In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install git+https://github.com/openai/whisper.git
!pip install pyannote.audio
!pip install torchaudio
!pip install pydub

In [ ]:

# 📁 Imports
from pydub import AudioSegment
from pyannote.audio import Pipeline
import whisper
import torch
import json
import os
from datetime import datetime


token = "XXXXXXXXXXXXXXXXXXXXXXXX"
#Remplacer

# 📍 Paramètres généraux
audio_path = "/content/drive/MyDrive/AudiosToTranscribe/Archive_001_LoganSamawithJME&NewhamGeneralsKissOct5th2010.mp3"  # à remplacer
archive_title = "LoganSamaNewhamGenerals"
mode_test = False


In [ ]:

# 🎧 Conversion en WAV mono 16kHz
audio = AudioSegment.from_mp3(audio_path)
wav = audio.set_channels(1).set_frame_rate(16000)

# 🎯 Mode test : extrait de 1:15 à 1:45 (30s)
if mode_test:
    start_ms = 75 * 1000
    end_ms = 105 * 1000
    clip = wav[start_ms:end_ms]
    wav_path = "test_clip.wav"
else:
    wav_path = "converted.wav"
    clip = wav

clip.export(wav_path, format="wav")
print(f"✅ Fichier audio exporté : {wav_path}")


In [ ]:

# 📌 Transcription avec Whisper
model = whisper.load_model("large")
result = model.transcribe(wav_path, word_timestamps=True)
segments = result["segments"]
print(f"✅ {len(segments)} segments transcrits")


In [ ]:

# 🧠 Diarisation
pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization", use_auth_token=token)

# 💡 Tu peux modifier ces valeurs selon ton cas
min_speakers = 2
max_speakers = 4

dz = pipeline(
    {"uri": "audio", "audio": wav_path},
    min_speakers=min_speakers,
    max_speakers=max_speakers
)
print("✅ Diarisation terminée")

# 🎙️ Indexation par temps
speaker_timeline = []
for turn, _, speaker in dz.itertracks(yield_label=True):
    speaker_timeline.append({
        "start": turn.start,
        "end": turn.end,
        "speaker": speaker
    })


In [ ]:

# 🧩 Construction du JSON avec découpage naturel (~5 mots)
final_data = {
    "title": archive_title,
    "instrumentals": [
        {"start": 0.0, "end": 60.0, "title": "PLACEHOLDER 1", "artist": "TODO"},
        {"start": 60.0, "end": 120.0, "title": "PLACEHOLDER 2", "artist": "TODO"},
        {"start": 120.0, "end": 180.0, "title": "PLACEHOLDER 3", "artist": "TODO"}
    ],
    "segments": []
}

def find_speaker(start, end):
    for s in speaker_timeline:
        if s["start"] <= start < s["end"]:
            return s["speaker"]
    return "unknown"

def split_by_diction(words, target_len=5, tolerance=3):
    max_words = target_len + tolerance
    min_words = max(1, target_len - tolerance)
    punctuation = ('.', '!', '?', ',', ';', '…')
    chunks = []
    current = []
    for word in words:
        w = word["word"].strip() if isinstance(word, dict) else word.word.strip()
        current.append(word)
        if len(current) >= max_words or w.endswith(punctuation):
            chunks.append(current)
            current = []
    if current:
        if len(current) < min_words and chunks:
            chunks[-1].extend(current)
        else:
            chunks.append(current)
    return chunks

segment_id = 0
for seg in segments:
    words = seg["words"]
    for word in words:
        word.pop("seek", None)
    chunks = split_by_diction(words)
    for chunk in chunks:
        segment = {
            "id": segment_id,
            "start": chunk[0]["start"],
            "end": chunk[-1]["end"],
            "text": ' '.join([w["word"] for w in chunk]),
            "words": chunk,
            "speaker": find_speaker(chunk[0]["start"], chunk[-1]["end"]),
            "instrumental": None
        }
        final_data["segments"].append(segment)
        segment_id += 1

# Export
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"archive_output_{timestamp}.json"
with open(filename, "w") as f:
    json.dump(final_data, f, indent=2)

print(f"✅ Fichier exporté : {filename}")
